In [1]:
import pandas as pd
import numpy as np

In [2]:
aucs = pd.read_csv('AUCs.txt', sep='\t')

In [3]:
aucs.head()

,Unnamed: 0,C4.5,C4.5+m,C4.5+cf,C4.5+m+cf
0,adult (sample),0.763,0.768,0.771,0.798
1,breast cancer,0.599,0.591,0.590,0.569
2,breast cancer wisconsin,0.954,0.971,0.968,0.967
3,cmc,0.628,0.661,0.654,0.657
4,ionosphere,0.882,0.888,0.886,0.898


Используя критерий знаковых рангов, проведите попарное сравнение каждого классификатора с каждым. Выберите два классификатора, различие между которыми наиболее статистически значимо.

In [4]:
from scipy.stats import wilcoxon

In [5]:
tests = []

for i, lhs_column in enumerate(aucs.columns[1:]):
    for j, rhs_column in enumerate(aucs.columns[1:]):
        if i >= j:
            continue
        
        test = wilcoxon(aucs[lhs_column], aucs[rhs_column])
        tests.append([lhs_column, rhs_column, test.statistic, test.pvalue])

In [6]:
cl_tests = pd.DataFrame.from_records(tests)
cl_tests.columns = ['model_A', 'model_B', 'statistic', 'p']

In [7]:
cl_tests

,model_A,model_B,statistic,p
0,C4.5,C4.5+m,6.5,0.010757
1,C4.5,C4.5+cf,43.0,0.861262
2,C4.5,C4.5+m+cf,11.0,0.015906
3,C4.5+m,C4.5+cf,17.0,0.046333
4,C4.5+m,C4.5+m+cf,22.0,0.327826
5,C4.5+cf,C4.5+m+cf,10.0,0.022909


Сравнивая 4 классификатора между собой, мы проверили 6 гипотез. Давайте сделаем поправку на множественную проверку. Начнём с метода Холма. Сколько гипотез можно отвергнуть на уровне значимости 0.05 после поправки этим методом?

In [19]:
from statsmodels.sandbox.stats.multicomp import multipletests

In [23]:
reject, p_corrected, a1, a2 = multipletests(cl_tests.p, 
                                            alpha = 0.05, 
                                            method = 'holm')

In [24]:
cl_tests['p_corrected'] = p_corrected
cl_tests['reject'] = reject

In [25]:
cl_tests

,model_A,model_B,statistic,p,p_corrected,reject
0,C4.5,C4.5+m,6.5,0.010757,0.064543,False
1,C4.5,C4.5+cf,43.0,0.861262,0.861262,False
2,C4.5,C4.5+m+cf,11.0,0.015906,0.079532,False
3,C4.5+m,C4.5+cf,17.0,0.046333,0.138998,False
4,C4.5+m,C4.5+m+cf,22.0,0.327826,0.655651,False
5,C4.5+cf,C4.5+m+cf,10.0,0.022909,0.091636,False


Сколько гипотез можно отвергнуть на уровне значимости 0.05 после поправки методом Бенджамини-Хохберга?

In [26]:
reject, p_corrected, a1, a2 = multipletests(cl_tests.p, 
                                            alpha = 0.05, 
                                            method = 'fdr_bh')

In [27]:
cl_tests['p_corrected'] = p_corrected
cl_tests['reject'] = reject

In [28]:
cl_tests

,model_A,model_B,statistic,p,p_corrected,reject
0,C4.5,C4.5+m,6.5,0.010757,0.045818,True
1,C4.5,C4.5+cf,43.0,0.861262,0.861262,False
2,C4.5,C4.5+m+cf,11.0,0.015906,0.045818,True
3,C4.5+m,C4.5+cf,17.0,0.046333,0.069499,False
4,C4.5+m,C4.5+m+cf,22.0,0.327826,0.393391,False
5,C4.5+cf,C4.5+m+cf,10.0,0.022909,0.045818,True


Насколько корректно, на ваш взгляд, применение метода Бенджамини-Хохберга в этой задаче?

Ответ: Некорректно: статистики, соответствующие разным гипотезам, зависимы, так что предположения метода Бенджамини-Хохберга не выполняются